In [1]:
import os
import glob
import joblib
import pandas as pd

from sklearn.metrics import roc_auc_score, log_loss, brier_score_loss, classification_report

# -----------------------------
# Paths
# -----------------------------
data_folder = "Processed_UE_Datasets_unscaled"

model_path = "threat_score_model.joblib"


# -----------------------------
# Load models
# -----------------------------
uncal_bundle = joblib.load(model_path)


uncal_model = uncal_bundle["model"]


feature_columns = uncal_bundle["feature_columns"]
label_column    = uncal_bundle["label_column"]

# -----------------------------
# Load TEST files only
# -----------------------------
csv_files = glob.glob(os.path.join(data_folder, "*.csv"))
test_files = [f for f in csv_files if "test" in os.path.basename(f).lower()]

if not test_files:
    raise RuntimeError("No test CSV files found.")

dfs = []
for f in test_files:
    df = pd.read_csv(f)
    missing = [c for c in feature_columns + [label_column] if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns {missing} in file: {os.path.basename(f)}")
    dfs.append(df)

test_data = pd.concat(dfs, axis=0).reset_index(drop=True)

X_test = test_data[feature_columns]
y_test = test_data[label_column]

print(f"Loaded {len(test_files)} test files")
print(f"Total test samples: {len(X_test)}")
print(f"Attack rate (test): {y_test.mean():.4f}")

# -----------------------------
# Predict probabilities
# -----------------------------
proba_uncal = uncal_model.predict_proba(X_test)[:, 1]

# -----------------------------
# Evaluation helper
# -----------------------------
def report(name, probs):
    preds = (probs >= 0.5).astype(int)
    print(f"\n=== {name} (TEST SET) ===")
    print("ROC AUC:  ", roc_auc_score(y_test, probs))
    print("Log Loss: ", log_loss(y_test, probs))
    print("Brier:    ", brier_score_loss(y_test, probs))
    print(classification_report(y_test, preds, digits=4))

# -----------------------------
# Compare
# -----------------------------
report("Results", proba_uncal)



Loaded 2 test files
Total test samples: 151991
Attack rate (test): 0.0307

=== Results (TEST SET) ===
ROC AUC:   0.9992101851269235
Log Loss:  0.008035268206489142
Brier:     0.0014314965773752073
              precision    recall  f1-score   support

           0     0.9997    0.9989    0.9993    147321
           1     0.9660    0.9904    0.9780      4670

    accuracy                         0.9986    151991
   macro avg     0.9828    0.9946    0.9887    151991
weighted avg     0.9987    0.9986    0.9986    151991



In [2]:
import sys
print(sys.executable)
print(sys.version)


/usr/local/opt/python@3.11/bin/python3.11
3.11.13 (main, Jun  3 2025, 18:38:25) [Clang 16.0.0 (clang-1600.0.26.6)]
